In [1]:
!conda create -n fast_unet python==3.10

/bin/bash: line 1: conda: command not found


In [2]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:16:06_PDT_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0


In [4]:
!python -m pip install torch torch-tensorrt tensorrt --extra-index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 26.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 KB 1.1 MB/s eta 0:00:00
donePreparing metadata (setup.py) ... 
doneing metadata (setup.py) ... 
doneing metadata (setup.py) ... 
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.5 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... one
done5h  Getting requirements to build wheel ... 
donePreparing metadata (pyproject.toml) ... 
doneng wheel for tensorrt (setup.py) ... 
  Created wheel for tensorrt: filename=tensorrt-10.7.0.post1-py2.py3-none-any.whl size=42186 sha256=185aa7fc2a685e92f9dc1313776be461976876afb3aeccba1fc02d5458a7c581
  Stored in directory: /root/.cache/pip/wheels/b1/3a/4a/7cbc5f10f993bc284e6db8bb2e72cac9f1b5811da7026f72e3
donesorrt-cu12 (setup.py) ... 
  Created wheel for tensorrt-cu12: filename=tensorrt_cu12-10.7.0.post1-py2

In [5]:
!pip install "nvidia-modelopt[all]" -U --extra-index-url https://pypi.nvidia.com

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.5/616.5 KB 13.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 KB 5.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 KB 1.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 21.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 28.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.5/291.5 MB 5.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 MB 27.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 KB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 8

In [6]:
!pip install onnx
!pip install onnxscript
!pip install timm optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.9/694.9 KB 13.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 86.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 KB 82.1 MB/s eta 0:00:00


In [7]:
!ls /workspace/onnx_models/fast_unet.onnx

fast_unet.onnx


In [1]:
import numpy as np
# Example numpy file for single-input ONNX
calib_data = np.random.randn(1, 1, 64, 256, 256)
calib_data = calib_data.astype(np.float32)
np.save("calib_data.npy", calib_data)

# Example numpy file for single/multi-input ONNX
# Dict key should match the input names of ONNX
# calib_data = {
#     "input_name": np.random.randn(*shape),
#     "input_name2": np.random.randn(*shape2),
# }
# np.savez("/workspace/calib_data.npz", calib_data)

import sys

sys.path.insert(0, './TensorRT-Model-Optimizer')

import modelopt.onnx.quantization as moq
import numpy as np

calibration_data_path = 'calib_data.npy'
# onnx_path = "vit_base_patch16_224.onnx"
#
calibration_data = np.load(calibration_data_path)

moq.quantize(
    onnx_path="/workspace/onnx_models/fast_unet_fp32.onnx",
    calibration_data=calibration_data,
    calibration_method='max',
    output_path="/workspace/onnx_models/quant_fast_unet_int8.onnx",
    quantize_mode="int8",
    high_precision_dtype="float32",
)

INFO:root:Model /workspace/onnx_models/fast_unet_fp32.onnx with opset_version 16 is loaded.
INFO:root:Quantization Mode: int8
INFO:root:Quantizable op types in the model: ['Conv']
INFO:root:Building non-residual Add input map ...
INFO:root:Searching for hard-coded patterns like MHA, LayerNorm, etc. to avoid quantization.
INFO:root:Building KGEN/CASK targeted partitions ...
INFO:root:Classifying the partition nodes ...
INFO:root:Total number of nodes: 91
2025-03-10 03:33:21.600564851 [W:onnxruntime:, graph.cc:1348 Graph] Initializer encoder.stages.0.0.convs.0.conv.weight appears in graph inputs and will not be treated as constant value/weight. This may prevent some of the graph optimizations, like const folding. Move it out of graph inputs if there is no need to override it, by either re-generating the model with latest exporter/converter or with the tool onnxruntime/tools/python/remove_initializer_from_input.py.
2025-03-10 03:33:21.600613622 [W:onnxruntime:, graph.cc:1348 Graph] Initia

In [16]:
!ls /workspace/onnx_models/

fast_unet.onnx	quant_fast_unet_fp16.engine


In [15]:
!rm -r /workspace/onnx_models/

In [19]:
!git clone https://github.com/NVIDIA/TensorRT-Model-Optimizer.git

Cloning into 'TensorRT-Model-Optimizer'...
remote: Enumerating objects: 4912, done.
remote: Counting objects: 100% (1078/1078), done.
remote: Compressing objects: 100% (811/811), done.
remote: Total 4912 (delta 678), reused 436 (delta 254), pack-reused 3834 (from 1)
Receiving objects: 100% (4912/4912), 23.68 MiB | 31.21 MiB/s, done.
Resolving deltas: 100% (3431/3431), done.


In [3]:
from modelopt.torch._deploy._runtime import RuntimeRegistry
from modelopt.torch._deploy.device_model import DeviceModel
from modelopt.torch._deploy.utils import OnnxBytes
# Configure deployment
deployment = {
    "runtime": "TRT",
    "version": "10.3",
    "precision": 'stronglyTyped',
}

# Create an ONNX bytes object
onnx_bytes = OnnxBytes('/workspace/onnx_models/quant_fast_unet_int8.onnx').to_bytes()

# Get the runtime client
client = RuntimeRegistry.get(deployment)

# Compile the TRT model
print("Compiling the TensorRT engine. This may take a few minutes...")
compiled_model = client.ir_to_compiled(onnx_bytes)
print("Compilation completed.")

# Print size of the compiled model
engine_size = len(compiled_model)
print(f"Size of the TensorRT engine: {engine_size / (1024 ** 2):.2f} MB")

# Create the device model
device_model = DeviceModel(client, compiled_model, metadata={})
print(f"Inference latency reported by device_model: {device_model.get_latency()} ms")

INFO:root:trtexec --onnx=/tmp/tmpm8e9bpr8/onnx/quant_fast_unet_int8.onnx --stronglyTyped --saveEngine=/tmp/tmpm8e9bpr8/quant_fast_unet_int8/quant_fast_unet_int8.engine --skipInference --builderOptimizationLevel=4 --verbose --exportLayerInfo=/tmp/tmpm8e9bpr8/quant_fast_unet_int8/quant_fast_unet_int8.engine.graph.json


Loading onnx model from path: /workspace/onnx_models/quant_fast_unet_int8.onnx
Compiling the TensorRT engine. This may take a few minutes...
Writing onnx model to path: /tmp/tmpm8e9bpr8/onnx


INFO:root:build_engine: 197180.02343177795 ms
INFO:root:trtexec --loadEngine=/tmp/tmp6akw0685/engine --warmUp=500 --avgRuns=100 --iterations=100 --dumpProfile --separateProfileRun --exportProfile=/tmp/tmp6akw0685/profile


Compilation completed.
Size of the TensorRT engine: 54.56 MB


INFO:root:profile_engine: 16576.071977615356 ms


Inference latency reported by device_model: 72.3352 ms


In [4]:
from modelopt.torch._deploy._runtime import RuntimeRegistry
from modelopt.torch._deploy.device_model import DeviceModel
from modelopt.torch._deploy.utils import OnnxBytes
# Configure deployment
deployment = {
    "runtime": "TRT",
    "version": "10.3",
    "precision": 'int8',
}

# Create an ONNX bytes object
onnx_bytes = OnnxBytes('/workspace/onnx_models/quant_fast_unet_int8.onnx').to_bytes()

# Get the runtime client
client = RuntimeRegistry.get(deployment)

# Compile the TRT model
print("Compiling the TensorRT engine. This may take a few minutes...")
compiled_model = client.ir_to_compiled(onnx_bytes)
print("Compilation completed.")

# Print size of the compiled model
engine_size = len(compiled_model)
print(f"Size of the TensorRT engine: {engine_size / (1024 ** 2):.2f} MB")

# Create the device model
device_model = DeviceModel(client, compiled_model, metadata={})
print(f"Inference latency reported by device_model: {device_model.get_latency()} ms")

Loading onnx model from path: /workspace/onnx_models/quant_fast_unet_int8.onnx
Compiling the TensorRT engine. This may take a few minutes...


INFO:root:trtexec --onnx=/tmp/tmpnvpafdr6/onnx/quant_fast_unet_int8.onnx --fp16 --int8 --saveEngine=/tmp/tmpnvpafdr6/quant_fast_unet_int8/quant_fast_unet_int8.engine --skipInference --builderOptimizationLevel=4 --verbose --exportLayerInfo=/tmp/tmpnvpafdr6/quant_fast_unet_int8/quant_fast_unet_int8.engine.graph.json


Writing onnx model to path: /tmp/tmpnvpafdr6/onnx


INFO:root:build_engine: 412503.8480758667 ms
INFO:root:trtexec --loadEngine=/tmp/tmpo3vxl8md/engine --warmUp=500 --avgRuns=100 --iterations=100 --dumpProfile --separateProfileRun --exportProfile=/tmp/tmpo3vxl8md/profile


Compilation completed.
Size of the TensorRT engine: 49.99 MB


INFO:root:profile_engine: 8237.624168395996 ms


Inference latency reported by device_model: 28.671385714285716 ms


In [5]:
from modelopt.torch._deploy._runtime import RuntimeRegistry
from modelopt.torch._deploy.device_model import DeviceModel
from modelopt.torch._deploy.utils import OnnxBytes
# Configure deployment
deployment = {
    "runtime": "TRT",
    "version": "10.3",
    "precision": 'fp32',
}

# Create an ONNX bytes object
onnx_bytes = OnnxBytes('/workspace/onnx_models/quant_fast_unet_int8.onnx').to_bytes()

# Get the runtime client
client = RuntimeRegistry.get(deployment)

# Compile the TRT model
print("Compiling the TensorRT engine. This may take a few minutes...")
compiled_model = client.ir_to_compiled(onnx_bytes)
print("Compilation completed.")

# Print size of the compiled model
engine_size = len(compiled_model)
print(f"Size of the TensorRT engine: {engine_size / (1024 ** 2):.2f} MB")

# Create the device model
device_model = DeviceModel(client, compiled_model, metadata={})
print(f"Inference latency reported by device_model: {device_model.get_latency()} ms")

Loading onnx model from path: /workspace/onnx_models/quant_fast_unet_int8.onnx
Compiling the TensorRT engine. This may take a few minutes...
Writing onnx model to path: /tmp/tmprg6zufbb/onnx


INFO:root:trtexec --onnx=/tmp/tmprg6zufbb/onnx/quant_fast_unet_int8.onnx --saveEngine=/tmp/tmprg6zufbb/quant_fast_unet_int8/quant_fast_unet_int8.engine --skipInference --builderOptimizationLevel=3 --verbose --exportLayerInfo=/tmp/tmprg6zufbb/quant_fast_unet_int8/quant_fast_unet_int8.engine.graph.json
INFO:root:build_engine: 65652.44674682617 ms
INFO:root:trtexec --loadEngine=/tmp/tmpo7jym83b/engine --warmUp=500 --avgRuns=100 --iterations=100 --dumpProfile --separateProfileRun --exportProfile=/tmp/tmpo7jym83b/profile


Compilation completed.
Size of the TensorRT engine: 54.09 MB


INFO:root:profile_engine: 16538.774490356445 ms


Inference latency reported by device_model: 73.03704285714286 ms


In [6]:
from modelopt.torch._deploy._runtime import RuntimeRegistry
from modelopt.torch._deploy.device_model import DeviceModel
from modelopt.torch._deploy.utils import OnnxBytes
# Configure deployment
deployment = {
    "runtime": "TRT",
    "version": "10.3",
    "precision": 'fp16',
}

# Create an ONNX bytes object
onnx_bytes = OnnxBytes('/workspace/onnx_models/quant_fast_unet_int8.onnx').to_bytes()

# Get the runtime client
client = RuntimeRegistry.get(deployment)

# Compile the TRT model
print("Compiling the TensorRT engine. This may take a few minutes...")
compiled_model = client.ir_to_compiled(onnx_bytes)
print("Compilation completed.")

# Print size of the compiled model
engine_size = len(compiled_model)
print(f"Size of the TensorRT engine: {engine_size / (1024 ** 2):.2f} MB")

# Create the device model
device_model = DeviceModel(client, compiled_model, metadata={})
print(f"Inference latency reported by device_model: {device_model.get_latency()} ms")

Loading onnx model from path: /workspace/onnx_models/quant_fast_unet_int8.onnx
Compiling the TensorRT engine. This may take a few minutes...
Writing onnx model to path: /tmp/tmpxgb0g9la/onnx


INFO:root:trtexec --onnx=/tmp/tmpxgb0g9la/onnx/quant_fast_unet_int8.onnx --fp16 --saveEngine=/tmp/tmpxgb0g9la/quant_fast_unet_int8/quant_fast_unet_int8.engine --skipInference --builderOptimizationLevel=3 --verbose --exportLayerInfo=/tmp/tmpxgb0g9la/quant_fast_unet_int8/quant_fast_unet_int8.engine.graph.json
INFO:root:build_engine: 115142.45390892029 ms
INFO:root:trtexec --loadEngine=/tmp/tmpjeaqfth7/engine --warmUp=500 --avgRuns=100 --iterations=100 --dumpProfile --separateProfileRun --exportProfile=/tmp/tmpjeaqfth7/profile


Compilation completed.
Size of the TensorRT engine: 49.02 MB


INFO:root:profile_engine: 8169.643402099609 ms


Inference latency reported by device_model: 28.815328571428573 ms


In [5]:
!wget https://developer.nvidia.com/downloads/compute/machine-learning/tensorrt/10.3.0/local_repo/nv-tensorrt-local-repo-ubuntu2204-10.3.0-cuda-12.5_1.0-1_amd64.deb

--2025-03-09 19:25:56--  https://developer.nvidia.com/downloads/compute/machine-learning/tensorrt/10.3.0/local_repo/nv-tensorrt-local-repo-ubuntu2204-10.3.0-cuda-12.5_1.0-1_amd64.deb
Resolving developer.nvidia.com (developer.nvidia.com)... 2.17.112.69, 2.17.112.73
Connecting to developer.nvidia.com (developer.nvidia.com)|2.17.112.69|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://developer.download.nvidia.com/compute/machine-learning/tensorrt/10.3.0/local_repo/nv-tensorrt-local-repo-ubuntu2204-10.3.0-cuda-12.5_1.0-1_amd64.deb [following]
--2025-03-09 19:25:57--  https://developer.download.nvidia.com/compute/machine-learning/tensorrt/10.3.0/local_repo/nv-tensorrt-local-repo-ubuntu2204-10.3.0-cuda-12.5_1.0-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 2.17.211.104, 84.53.132.89, 84.53.132.136, ...
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|2.17.211.104|:443... conn

In [7]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.5 LTS
Release:	22.04
Codename:	jammy


In [ ]:
os="ubuntuxx04"
tag="10.x.x-cuda-x.x"

In [18]:
!sudo dpkg -i nv-tensorrt-local-repo-ubuntu2204-10.3.0-cuda-12.5_1.0-1_amd64.deb

(Reading database ... 40294 files and directories currently installed.)
Preparing to unpack nv-tensorrt-local-repo-ubuntu2204-10.3.0-cuda-12.5_1.0-1_amd64.deb ...
Unpacking nv-tensorrt-local-repo-ubuntu2204-10.3.0-cuda-12.5 (1.0-1) over (1.0-1) ...
Setting up nv-tensorrt-local-repo-ubuntu2204-10.3.0-cuda-12.5 (1.0-1) ...


In [9]:
!sudo cp /var/nv-tensorrt-local-repo-ubuntu2204-10.3.0-cuda-12.5/nv-tensorrt-local-620E7D29-keyring.gpg /usr/share/keyrings/

In [7]:
!/usr/src/tensorrt/bin/trtexec

&&&& RUNNING TensorRT.trtexec [TensorRT v100900] [b34] # /usr/src/tensorrt/bin/trtexec
=== Model Options ===
  --onnx=<file>               ONNX model

=== Build Options ===
  --minShapes=spec                   Build with dynamic shapes using a profile with the min shapes provided
  --optShapes=spec                   Build with dynamic shapes using a profile with the opt shapes provided
  --maxShapes=spec                   Build with dynamic shapes using a profile with the max shapes provided
  --minShapesCalib=spec              Calibrate with dynamic shapes using a profile with the min shapes provided
  --optShapesCalib=spec              Calibrate with dynamic shapes using a profile with the opt shapes provided
  --maxShapesCalib=spec              Calibrate with dynamic shapes using a profile with the max shapes provided
                                     Note: All three of min, opt and max shapes must be supplied.
                                           However, if only opt shape

In [8]:
!ls /usr/

cudnn_samples_v8  python3.10


In [9]:
!sudo apt-get update

Get:1 file:/var/nv-tensorrt-local-repo-ubuntu2204-10.3.0-cuda-12.5  InRelease [1572 B]
Get:1 file:/var/nv-tensorrt-local-repo-ubuntu2204-10.3.0-cuda-12.5  InRelease [1572 B]
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease               
Get:3 file:/var/nv-tensorrt-local-repo-ubuntu2204-10.3.0-cuda-12.5  Packages [5343 B]
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease   
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Reading package lists... Done


In [22]:
!sudo apt-get -y install tensorrt

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libnvinfer-dev libnvinfer-dispatch-dev libnvinfer-headers-plugin-dev
  libnvinfer-lean-dev libnvinfer-plugin-dev libnvinfer-samples
  libnvinfer-vc-plugin-dev libnvonnxparsers-dev python3-libnvinfer
  python3-libnvinfer-dev python3-libnvinfer-dispatch python3-libnvinfer-lean
The following NEW packages will be installed:
  libnvinfer-dispatch-dev libnvinfer-headers-plugin-dev libnvinfer-lean-dev
  libnvinfer-plugin-dev libnvinfer-samples libnvinfer-vc-plugin-dev
  libnvonnxparsers-dev python3-libnvinfer python3-libnvinfer-dev
  python3-libnvinfer-dispatch python3-libnvinfer-lean tensorrt
The following packages will be upgraded:
  libnvinfer-dev
1 upgraded, 12 newly installed, 0 to remove and 52 not upgraded.
9 not fully installed or removed.
Need to get 0 B/2459 MB of archives.
After this operation, 6656 MB of additional disk space will 

In [7]:
!dpkg-query -W tensorrt

tensorrt	10.9.0.34-1+cuda12.8


In [2]:
!ln -s /usr/src/tensorrt/bin/trtexec /usr/local/bin/trtexec

In [8]:
!trtexec

&&&& RUNNING TensorRT.trtexec [TensorRT v100300] # trtexec
=== Model Options ===
  --onnx=<file>               ONNX model

=== Build Options ===
  --minShapes=spec                   Build with dynamic shapes using a profile with the min shapes provided
  --optShapes=spec                   Build with dynamic shapes using a profile with the opt shapes provided
  --maxShapes=spec                   Build with dynamic shapes using a profile with the max shapes provided
  --minShapesCalib=spec              Calibrate with dynamic shapes using a profile with the min shapes provided
  --optShapesCalib=spec              Calibrate with dynamic shapes using a profile with the opt shapes provided
  --maxShapesCalib=spec              Calibrate with dynamic shapes using a profile with the max shapes provided
                                     Note: All three of min, opt and max shapes must be supplied.
                                           However, if only opt shapes is supplied then it will b

In [9]:
!wget https://developer.nvidia.com/downloads/compute/machine-learning/tensorrt/10.3.0/tars/TensorRT-10.3.0.26.Linux.x86_64-gnu.cuda-12.5.tar.gz

--2025-03-10 02:41:34--  https://developer.nvidia.com/downloads/compute/machine-learning/tensorrt/10.3.0/tars/TensorRT-10.3.0.26.Linux.x86_64-gnu.cuda-12.5.tar.gz
Resolving developer.nvidia.com (developer.nvidia.com)... 2.17.35.66, 2.17.35.153
Connecting to developer.nvidia.com (developer.nvidia.com)|2.17.35.66|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://developer.download.nvidia.com/compute/machine-learning/tensorrt/10.3.0/tars/TensorRT-10.3.0.26.Linux.x86_64-gnu.cuda-12.5.tar.gz [following]
--2025-03-10 02:41:35--  https://developer.download.nvidia.com/compute/machine-learning/tensorrt/10.3.0/tars/TensorRT-10.3.0.26.Linux.x86_64-gnu.cuda-12.5.tar.gz
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 2.18.188.11, 2.18.188.32
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|2.18.188.11|:443... connected.
200 OKequest sent, awaiting response... 
Length: 4381146404 (4.1G) [applicatio

In [14]:
%%capture

!tar -xvzf TensorRT-10.3.0.26.Linux.x86_64-gnu.cuda-12.5.tar.gz

In [18]:
!ln -sf /home/TensorRT-10.3.0.26/bin/trtexec /usr/local/bin/trtexec

In [19]:
!TENSORRT_PATH=$(pwd)/TensorRT-10.3.0.26

In [20]:
!echo "export TENSORRT_HOME=${TENSORRT_PATH}" >> ~/.bashrc
!echo "export PATH=\$TENSORRT_HOME/bin:\$PATH" >> ~/.bashrc
!echo "export LD_LIBRARY_PATH=\$TENSORRT_HOME/lib:\$LD_LIBRARY_PATH" >> ~/.bashrc

In [21]:
!source ~/.bashrc

In [22]:
!cd TensorRT-10.3.0.26/python/
!pip install ./tensorrt-10.3.0-cp39-none-linux_x86_64.whl
!cd ../..

ERROR: tensorrt-10.3.0-cp39-none-linux_x86_64.whl is not a supported wheel on this platform.


In [23]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 KB 3.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 KB 6.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.2/149.2 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 KB 12.0 MB/s eta 0:00:00
doneing metadata (setup.py) ... 
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 15.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.4/899.4 KB 21.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 KB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.2/829.2 KB 28.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 10.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 KB 29.0 MB/s eta 0:00: